# 03 - Grid / Block Model
Proposito: generar la grilla de bloques desde config o extents.

Inputs:
- `config/project.json`
- CSV en `cfg["data_csv_path"]`

Outputs esperados:
- `outputs/tables/grid.csv`


### 1. Project setup


In [ ]:
import os, sys, json, glob
from IPython.display import Image, display

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir(PROJECT_ROOT)
else:
    PROJECT_ROOT = os.getcwd()
    os.chdir(PROJECT_ROOT)

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("CWD:", os.getcwd())

from src.io_utils import read_csv_robust, standardize_columns, coerce_numeric
from src.grid import grid_from_extents, make_grid_dataframe, export_grid_to_csv
from src.make_demo_data import make_demo_csv


### 2. Load config


In [ ]:
cfg_path = "config/project.json"
print("Config:", os.path.abspath(cfg_path))
with open(cfg_path, "r", encoding="utf-8") as f:
    cfg = json.load(f)

print("Data CSV:", cfg["data_csv_path"])
mapping = {
    "x": cfg["columns"].get("x"),
    "y": cfg["columns"].get("y"),
    "z": cfg["columns"].get("z"),
    "var": cfg["columns"].get("variable_objetivo"),
}
print("Column mapping:", mapping)


### 3. Load data


In [ ]:
if not os.path.exists(cfg["data_csv_path"]):
    make_demo_csv(cfg["data_csv_path"])

df_raw = read_csv_robust(cfg["data_csv_path"])
df = standardize_columns(df_raw, mapping)
coerce_numeric(df, ["x", "y", "z", "var"])

rows_before = len(df)
df.dropna(subset=["x", "y", "var"], inplace=True)
rows_after = len(df)
removed_pct = 0.0 if rows_before == 0 else (rows_before - rows_after) * 100.0 / rows_before

print("Shape raw:", df_raw.shape, "clean:", df.shape)
print("Dtypes:
", df.dtypes)
print("Rows removed (%):", f"{removed_pct:.2f}")
print("X range:", (df["x"].min(), df["x"].max()))
print("Y range:", (df["y"].min(), df["y"].max()))
if "z" in df.columns and df["z"].notna().any():
    print("Z range:", (df["z"].min(), df["z"].max()))
else:
    print("Z range: n/a")


### 4. Build grid


In [ ]:
grid_cfg = cfg.get("grid", {})
if grid_cfg.get("auto_from_data", True):
    grid_spec = grid_from_extents(
        df,
        "x",
        "y",
        "z",
        float(grid_cfg.get("dx", 25.0)),
        float(grid_cfg.get("dy", 25.0)),
        float(grid_cfg.get("dz", 5.0)),
        pad=float(grid_cfg.get("pad", 0.0)),
    )
else:
    grid_spec = {
        "nx": grid_cfg["nx"],
        "ny": grid_cfg["ny"],
        "nz": grid_cfg.get("nz", 1),
        "xmin": grid_cfg["xmin"],
        "ymin": grid_cfg["ymin"],
        "zmin": grid_cfg.get("zmin", 0.0),
        "dx": grid_cfg["dx"],
        "dy": grid_cfg["dy"],
        "dz": grid_cfg.get("dz", 1.0),
    }

grid_df = make_grid_dataframe(grid_spec)
os.makedirs("outputs/tables", exist_ok=True)
export_grid_to_csv(grid_df, "outputs/tables/grid.csv")

print(grid_df.head())


### 5. Artifacts generated


In [ ]:
figure_paths = sorted(glob.glob("outputs/figures/*.png"))
table_paths = sorted(glob.glob("outputs/tables/*.csv"))
model_paths = sorted(glob.glob("outputs/models/*"))

print("Figures:", [os.path.abspath(p) for p in figure_paths])
print("Tables:", [os.path.abspath(p) for p in table_paths])
print("Models:", [os.path.abspath(p) for p in model_paths])

for p in figure_paths:
    display(Image(filename=p))
